## Week 2 Day 3

Now we get to more detail:

1. Different models

2. Structured Outputs

3. Guardrails

In [4]:
from dotenv import load_dotenv
from openai import AsyncAzureOpenAI
from openai import AsyncOpenAI
from agents import (
    set_default_openai_client,
    set_default_openai_api,
    set_tracing_disabled,
    Agent,
    Runner,
    trace,
    function_tool,
    OpenAIChatCompletionsModel,
    input_guardrail,
    GuardrailFunctionOutput
)
# from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [5]:
load_dotenv(override=True)

True

In [7]:
# Set up Azure OpenAI client
azure_client = AsyncAzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
)

# Set this client as the default for the Agents SDK
set_default_openai_client(azure_client)
set_default_openai_api("chat_completions") # is needed for the Agents SDK with Azure at the moment
set_tracing_disabled(False) # is needed for the Agents SDK with Azure at the moment


# Define the model — must match your Azure deployment name
azure_model_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")

openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
DeepSeek API Key not set (and this is optional)
Groq API Key exists and begins gsk_


In [8]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

### It's easy to use any models with OpenAI compatible endpoints

In [9]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
AZURE_BASE_URL = "https://openai-api.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-15-preview"

In [10]:

# deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

#deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [11]:
sales_agent1 = Agent(name="Azure Sales Agent", instructions=instructions1, model=azure_model_deployment)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

In [12]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [17]:
sender_email = os.getenv("FROM_EMAIL")
recipient_email = os.getenv("TO_EMAIL")

@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(sender_email)  # Change to your verified sender
    to_email = To(recipient_email)  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.send(mail)
    return {"status": "success"}

In [19]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model=azure_model_deployment)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model=azure_model_deployment)
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [20]:
email_tools = [subject_tool, html_tool, send_html_email]

In [21]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model=azure_model_deployment,
    handoff_description="Convert an email to HTML and send it")

In [22]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [24]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=azure_model_deployment)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

## Check out the trace:

https://platform.openai.com/traces

In [25]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model=azure_model_deployment
)

In [26]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [28]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model=azure_model_deployment,
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [ ]:
# Safe run with guardrail handling and Azure deployment
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model=azure_model_deployment,
    input_guardrails=[guardrail_against_name]
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

try:
    with trace("Protected Automated SDR"):
        result = await Runner.run(careful_sales_manager, message)
        print(result)
except Exception as e:
    if e.__class__.__name__ in ("InputGuardrailTripwireTriggered", "GuardrailTripwireTriggered"):
        print("Input blocked by guardrail (personal name detected). Removing the personal name and retrying...")
        # Derive safe_message by removing the detected personal name from the original message
        name_check = await Runner.run(guardrail_agent, message)
        detected_name = getattr(getattr(name_check, "final_output", None), "name", None)
        if detected_name:
            import re
            pattern = re.compile(re.escape(detected_name), flags=re.IGNORECASE)
            safe_message = pattern.sub("", message).replace("  ", " ").strip()
        else:
            safe_message = message
        with trace("Protected Automated SDR (retry)"):
            result = await Runner.run(careful_sales_manager, safe_message)
            print(result)
    else:
        raise


Input blocked by guardrail (personal name detected). Removing the personal name and retrying...


## Check out the trace:

https://platform.openai.com/traces

In [42]:

message = "Send out a cold sales email addressed to the CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)
print(result)

RunResult:
- Last agent: Agent(name="Sales Manager", ...)
- Final output (str):
    Based on the three generated emails, I recommend selecting the following option as it's engaging, conversational, and impactful:
    
    ---
    
    **Subject: Is your SOC 2 compliance making you want to SOC it to someone else? (Spoiler: We can help!)**
    
    Hey [CEO Name],
    
    Let's be honest, running a company is like juggling flaming chainsaws while riding a unicycle. You've got product, sales, growth, and then BAM! Security compliance rears its ugly head. And if that head is SOC 2, well, you might as well add 'lion tamer' to that unicycle act.
    
    That's where ComplAI comes in. We're like the pit crew for your compliance race. We've built an AI-powered SaaS platform that *automates* the drudgery of SOC 2, turning the mountain of evidence gathering into a molehill of "done."
    
    Think of us as your secret weapon against audit anxiety. We help you:
    
    * **Automate Evidence C

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Try different models<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation
            </span>
        </td>
    </tr>
</table>